# 9. A Loan Data Practical Example with NumPy

[1. Setting Up: Importing the Data Set](#01)    
[2. Setting Up: Checking for Incomplete Data](#02)    
[6. Manipulating Text Data: Loan Status and Term](#06)    
[7. Manipulating Text Data: Grade and Sub Grade](#07)

[8. Manipulating Text Data: Verification Status & URL](#08)

[9. Manipulating Text Data: State Address](#09)

[10. Manipulating Text Data: Converting Strings and Creating a Checkpoint](#10)

[## 11. Manipulating Numeric Data: Substitute Filler Values](#11)

[JM Walrus Operator Discover](#JM-Walrus-Operator-Discover)

Setting Up: Introduction to the Practical Example
Setting Up: Importing the Data Set
Setting Up: Checking for Incomplete Data
Setting Up: Splitting the Dataset
Setting Up: Creating Checkpoints
Manipulating Text Data: Issue Date
Manipulating Text Data: Loan Status and Term
Manipulating Text Data: Grade and Sub Grade
Manipulating Text Data: Verification Status & URL
Manipulating Text Data: State Address
Manipulating Text Data: Converting Strings and Creating a Checkpoint
Manipulating Numeric Data: Substitute Filler Values
Manipulating Numeric Data: Currency Change - The Exchange Rate
Manipulating Numeric Data: Currency Change - From USD to EUR|

## 0. Importing Libraries and Defining Functions

- NumPy library, show version, set_printoptions to better see the data
- def show_attr(arrnm): to view arrays attributes: shape, ndim, size, dtype.
- def chkpt(filenm, chk_header, chk_data): to create Checkpoints for the process.

In [1]:
import numpy as np
np.__version__
np.set_printoptions(suppress=True, linewidth=100, precision=2)

In [2]:
# Function show_attr

def show_attr(arrnm: str) -> str:
    strout = f' {arrnm}: '

    for attr in ('shape', 'ndim', 'size', 'dtype'):     #, 'itemsize'):
            arrnm_attr = arrnm + '.' + attr
            strout += f'| {attr}: {eval(arrnm_attr)} '

    return strout

In [3]:
# Function show_thseps

def show_thseps(num: float, underscore: bool = False) -> str:
    '''Insert thousand separators'''
    
    sep = ','
    if underscore:
        sep = '_'

    n_str = str(num)
    dec_ix = n_str.find('.')
    if dec_ix > -1:
        dec_part = n_str[dec_ix:]
    else:
        dec_part = ''

    int_pr = n_str[:dec_ix][::-1]
    int_part = ''
    for i in range(len(int_pr)):
        int_part += int_pr[i]
        if (i + 1) % 3 == 0:
            int_part += sep
    
    return int_part[::-1] + dec_part    

In [4]:
# Function chkpt (checkpoint)

def chkpt(filenm: str, chk_header: np.ndarray,
          chk_data: np.ndarray) -> np.lib.npyio.NpzFile:
    np.savez(filenm, header=chk_header, data=chk_data)
    chkpt_var = np.load(f'{filenm}.npz')
    return chkpt_var

<a id="01"></a>
## 1. Setting Up: Importing the Data Set

### Quick glance at loan-data.csv (notepad++)
- Contains both text and numeric data.
- A header clarifying the contents of each column
- 1st col is called 'id' -> each row consists of info for the account of a loan candidate's application and each candidate is described by their id. => We referred to the rows as accounts, candidates or applications.
- Can't see whether there are missing values in the dtset.
- ';' as delimiter

### genfromtxt(autostrip=False)
- autostrip: bool, optional
Whether to automatically strip white spaces from the variables.

In [5]:
raw_data_np = np.genfromtxt('9_loan-data.csv',
                            delimiter=';',
                            skip_header=1,
                            autostrip=True)
display(raw_data_np)
display(show_attr('raw_data_np'))
print('Num of NANs:', np.isnan(raw_data_np).sum())

# Lot of NANs, either text or missing
# The entire 1st row is NAN so the skip_header=1
# autostrip cause it removes white spaces which can distort our cols

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

' raw_data_np: | shape: (10000, 14) | ndim: 2 | size: 140000 | dtype: float64 '

Num of NANs: 88005


<a id="02"></a>
## 2. Setting Up: Checking for Incomplete Data

- I calc NANs num early: 88_005
- Also calc and store temp_fill, orig_means, and orig_stats[mins, means, maxs] <- over Cols>

In [6]:
# We have 88_005 NANs to take care
# 1st calc the orig_means of e/col to have an idea

temp_fill = np.nanmax(raw_data_np) + 1          # *Note*
orig_means = np.nanmean(raw_data_np, axis=0)
display(temp_fill, orig_means)
# .py:3: RuntimeWarning: Mean of empty slice
# 8 cols [1,3,5,8-12] full of NAN (empty or strings)

display(orig_means[[1,3,5,8,9,10,11,12]])
np.isnan(orig_means).sum()

# *Note*: it not necessary, there are faster 'Replacement NANs' methods

C:\Users\jm\AppData\Local\Temp\ipykernel_9752\4186509109.py:5: RuntimeWarning: Mean of empty slice
  orig_means = np.nanmean(raw_data_np, axis=0)


68616520.0

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

array([nan, nan, nan, nan, nan, nan, nan, nan])

8

#### Split data in next steps.
- NumPy doesn't like when we store multiple type of data like numbers and text in the same array.
- Since it limits what we can do with the dtset.
- We should split the data into two smaller arrays:
    1. One containing the numeric values
    2. and another one the strings
> Before we do so we should extract the minimum and maximum values for each numeric columm

In [7]:
# Before split (num / strings) calc min and max for each col and store 
orig_stats = np.array([np.nanmin(raw_data_np, axis=0),
                      orig_means,
                      np.nanmax(raw_data_np, axis=0)])
orig_stats
# Of course two warnings for the entire NANs in calcs of min and max
# We'll use later to fill missing with the worst case.

C:\Users\jm\AppData\Local\Temp\ipykernel_9752\2288172437.py:2: RuntimeWarning: All-NaN slice encountered
  orig_stats = np.array([np.nanmin(raw_data_np, axis=0),
C:\Users\jm\AppData\Local\Temp\ipykernel_9752\2288172437.py:4: RuntimeWarning: All-NaN slice encountered
  np.nanmax(raw_data_np, axis=0)])


array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

## 3. Splitting the Dataset

- An data_str and data num. Reload separated arrays. Check NANs
- Also fill NANs in data num. Also load header_str and header_num

In [8]:
# Get the cols_str(ixs), and the cols_num(ixs)
# cols_str = np.argwhere(np.isnan(orig_means)).reshape(8,)
cols_str = np.argwhere(np.isnan(orig_means)).squeeze()
display(cols_str)

# cols_num = np.argwhere(~np.isnan(orig_means)).squeeze()
cols_num = np.argwhere(np.isnan(orig_means) == False).squeeze()
display(cols_num)

# Check: num_cols_str + num_cols_num = raw_data.shape[1]
len(cols_str) + len(cols_num) == raw_data_np.shape[1]

array([ 1,  3,  5,  8,  9, 10, 11, 12], dtype=int64)

array([ 0,  2,  4,  6,  7, 13], dtype=int64)

True

In [9]:
# Reload the data in two dtsets dividing cols in str and num w/usecols
## JM no necessary to reload de data_num, we can filter existing data

data_str = np.genfromtxt('9_loan-data.csv',
                         delimiter=';',
                         skip_header=1,
                         usecols=cols_str,
                         dtype=str)
display(data_str)
display(show_attr('data_str'))
# TypeError: np.isnan in str dtype, then 
print("Num of Empties (''):", data_str[data_str == ""].size)

array([['May-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', ' 36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

' data_str: | shape: (10000, 8) | ndim: 2 | size: 80000 | dtype: <U69 '

Num of Empties (''): 3529


In [10]:
# Reload the num part of the dtset filling NANs

data_num = np.genfromtxt('9_loan-data.csv',
                         delimiter=';',
                         skip_header=1,
                         usecols=cols_num,
                         filling_values=temp_fill)
display(data_num)
display(show_attr('data_num'))
print("Num of NANs:", np.isnan(data_num).sum())

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

' data_num: | shape: (10000, 6) | ndim: 2 | size: 60000 | dtype: float64 '

Num of NANs: 0


In [11]:
# JM way to get data_num without re-read file
data_num_jm = raw_data_np[:,cols_num]
display(np.isnan(data_num_jm).sum())
data_num_jm[np.nonzero(np.isnan(data_num_jm))] = temp_fill
# We could have left the NANs and at the time of replacing make the condition (np.isnan())

display(data_num_jm)
display(show_attr('data_num_jm'))
print("Num of NANs:", np.isnan(data_num_jm).sum())
print('data_num == data_num_jm:', np.array_equal(data_num, data_num_jm))

8005

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

' data_num_jm: | shape: (10000, 6) | ndim: 2 | size: 60000 | dtype: float64 '

Num of NANs: 0
data_num == data_num_jm: True


In [12]:
## The Names of the Columns - get header full
header_full = np.genfromtxt('9_loan-data.csv',
                           delimiter=';',
                           skip_footer=raw_data_np.shape[0],
                           dtype=str)
display(header_full)
display(show_attr('header_full'))
# TypeError: np.isnan in str dtype, then 
print("Num of Empties (''):", header_full[header_full == ""].size)

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

' header_full: | shape: (14,) | ndim: 1 | size: 14 | dtype: <U19 '

Num of Empties (''): 0


In [13]:
### JM way: get only 1st row (data[0])
header_full_jm = np.genfromtxt('9_loan-data.csv',
                              delimiter=';',
                              dtype=str)[0]
display(header_full_jm)
display(show_attr('header_full_jm'))
# TypeError: np.isnan in str dtype, then 
print("Num of Empties (''):", header_full_jm[header_full_jm == ""].size)

print('header_full == header_full_jm:', np.array_equal(header_full, header_full_jm))

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U69')

' header_full_jm: | shape: (14,) | ndim: 1 | size: 14 | dtype: <U69 '

Num of Empties (''): 0
header_full == header_full_jm: True


In [14]:
# Split header_full in header_str and header_num
header_str, header_num = header_full[cols_str], header_full[cols_num]
display(header_str, header_num)

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

## 4. Creating Checkpoints

> Before we start manipulating the text data, we want to briefly discuss the practice of using checkpoints and how will implement it.

#### Checkpoints:
- Places throughout our code where we store a copy of our dataset (or only parts of it).
- Te idea is that we want to avoid losing a lot of progress if we accidentally override the variables we've been working with.
- This is an extremely reliable practice when we need to clean or pre-process many parts of a dtset.
- Because we're creating a failsafe we can rely on.
- def chkpt function in section 0.

In [15]:
chkpt_test = chkpt('chkpt-test', header_str, data_str)
print('type(chkpt_test):', type(chkpt_test))
display(chkpt_test['header'])
display(chkpt_test['data'])
type(chkpt_test['header'])

type(chkpt_test): <class 'numpy.lib.npyio.NpzFile'>


array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

array([['May-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', ' 36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

numpy.ndarray

In [16]:
# chkpt_test['data'] idem data_str in this case
np.array_equal(chkpt_test['data'], data_str)

True

## 5. Manipulating Text Data: Issue Date

- We have a column (issue_d). Chack: 9_LoanDatasetDictionary.xlsx
- issue_d: The month which the loan was funded. Change to issue_date (more descriptive)
- The values of this col are strings (all are -15, year 2015)
- We're going to convert characters (str) to numbers(str), eliminating -15 and converting months_names to months_mum.

In [17]:
# 0. check the names of the cols in header_str
display(header_str)
header_str[0]

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

'issue_d'

In [18]:
# 1. Transform col name to issue_date (more descriptive)
header_str[0] = 'issue_date'
header_str

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [19]:
# 2. check the unique values of issue_date col and
# eliminate '-15' (all are year 2015)
display(np.unique(data_str[:,0]))
data_str[:,0] = np.chararray.strip(data_str[:,0], '-15')
display(np.unique(data_str[:,0]))
data_str[:,0]

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'],
      dtype='<U69')

array(['May', '', 'Sep', ..., 'Jun', 'Apr', 'Dec'], dtype='<U69')

In [20]:
# 3. Replace the month_name by month_num
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec'])

for i in range(13):
    data_str[:,0] = np.where(data_str[:,0] == months[i],
                             i,
                             data_str[:,0])
    
display(np.unique(data_str[:,0]))
data_str[:,0]
# We won't cast the values to int right away cause we're going to
# process al str cols and convert at the end the entire dtset

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

array(['5', '0', '9', ..., '6', '4', '12'], dtype='<U69')

<a id="06"></a>
## 6. Manipulating Text Data: Loan Status and Term

- Continuing examining our str variables (cols) we find 'Loan Status' and 'Term'.
- loan_status: we've been told that regressions that determine the probability of default only care if the candidate is in a stable financial condition.
- loan_status: should be a simple dummy indicator of whether the applicant is in a good or bad economic state.
- loan_status: 'Current', 'Fully Paid', 'In Grace Period', 'Issued',  'Late (16-30 days)' => GOOD = 1 (positive fashion)
- loan_status: '', 'Charged Off', 'Default', 'Late (31-120 days)' => BAD = 0 ('' = 0, risk-averse, worst case)'
- term: only 3 unique vals: '', ' 36 months', ' 60 months'. We can strip ' months'.
- term_months: changed the name and preprocessing only will have 2 vals 36 ans 60.
- term_months: when we only have two possible numerical outcomes for a given col -> we could use 1 and 0 instead. Then ask somewhere. Nothing to ask we'll assume that there is a significance to these values.

#### numpy.isin(element, test_elements, assume_unique=False, invert=False, *, kind=None)
- Calculates element in test_elements, broadcasting over element only. Returns a boolean array of the same shape as element that is True where an element of element is in test_elements and False otherwise.

In [21]:
# 0. Let's look at the variables that follow issue_date
display(header_str)
display(header_str[[1,2]], header_str[1:3])

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

array(['loan_status', 'term'], dtype='<U19')

array(['loan_status', 'term'], dtype='<U19')

In [22]:
# 1a. Let's look the data in 'loan_status' 
display(data_str[:,1])
display(np.unique(data_str[:,1]))
show_attr('np.unique(data_str[:,1])')
# 9 diff vals for individual els of this col

array(['Current', 'Current', 'Current', ..., 'Current', 'Current', 'Current'], dtype='<U69')

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

' np.unique(data_str[:,1]): | shape: (9,) | ndim: 1 | size: 9 | dtype: <U69 '

In [23]:
# 2a. Replace the vals for 1 or 0 if their are GOOD or BAD loan_status
bad = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])
data_str[:,1] = np.where(np.isin(data_str[:,1], bad), 0, 1)
# Check
display(data_str[:,1])
display(np.unique(data_str[:,1]))

array(['1', '1', '1', ..., '1', '1', '1'], dtype='<U69')

array(['0', '1'], dtype='<U69')

In [24]:
# 1b. Let's look the data in 'term' 
display(data_str[:,2])
display(np.unique(data_str[:,2]))
show_attr('np.unique(data_str[:,2])')
# 3 diff vals for individual els of this col

array([' 36 months', ' 36 months', ' 36 months', ..., ' 36 months', ' 36 months', ' 36 months'],
      dtype='<U69')

array(['', ' 36 months', ' 60 months'], dtype='<U69')

' np.unique(data_str[:,2]): | shape: (3,) | ndim: 1 | size: 3 | dtype: <U69 '

In [25]:
# 2b. strip ' months' and change col name to indicates num are months
data_str[:,2] = np.chararray.strip(data_str[:,2], ' months')
display(data_str[:,2])
display(np.unique(data_str[:,2]))

# change col name to indicates num are months
header_str[2] = 'term_months'
header_str

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

array(['', '36', '60'], dtype='<U69')

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [26]:
# 3b. assign 60 (worst case) to empties
data_str[:,2] = np.where(data_str[:,2] == '',
                         '60',                # assume the worst
                         data_str[:,2])
np.unique(data_str[:,2])

array(['36', '60'], dtype='<U69')

<a id="07"></a>
## 7. Manipulating Text Data: Grade and Sub Grade

- Next to columns 'grade' and 'sub_grade'
- Uniques 'grade': '', 'A', 'B', 'C', 'D', 'E', 'F', 'G'
- Uniques 'sub_grade': '', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4','C5', 'D1','D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2','F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5'
- For e/el we saw in grade, we have 5 diff els in sub_grade
- If the data is complete any info we can get from 'grade' col can also be obtained from the 'sub_grade' col.
- That in theory makes 'grade' redundant (not always the case).
- When there are missing els in sub_grade, we can use grade to assign more appropriate estimations.
- Ex. if the sub_grade for an account is missing but the grade is B, it makes more sense to assign to ir a sub_grade B5 rather than the worst possible grade (G5).
- After replacing the '' in sub_grade (referring grade values) we still have 9 of them left. 9 in 10000 is relatively nothing. We could drop this vals. 
- There is something better we could do (our job requires us to assign status to every individual): create a whole new category lower than G5 (precautionary step cause accounts which are withholding info must be penalized accordingly).
- The info in grade will stay in sub_grade then we can remove 'grade' col.
- Then we'll replace categorical(s) of sub_grade by numbers using a dic constructed w/zip Python function.

#### zip(*iterables)
- Make an iterator that aggregates elements from each of the iterables.
- Returns an iterator of tuples, where the i-th tuple contains the i-th element from each of the argument sequences or iterables. The iterator stops when the shortest input iterable is exhausted. With a single iterable argument, it returns an iterator of 1-tuples. With no arguments, it returns an empty iterator. Equivalent to:

In [27]:
# 0. Let's see the headers and contents of grade and sub_grade
display(header_str)
display(header_str[3:5])

display(data_str[:,3:5])    # same display(data_str[:,[3,4]])
# display(np.unique(data_str[:,3]))
# display(np.unique(data_str[:,4]))

for i in (3,4):
    uniques = np.unique(data_str[:,i])
    print(f"{'-' * 30}\n{uniques.size} uniques of '{header_str[i]}':")
    display(uniques)


array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

array(['grade', 'sub_grade'], dtype='<U19')

array([['C', 'C3'],
       ['A', 'A5'],
       ['B', 'B5'],
       ...,
       ['A', 'A5'],
       ['D', 'D2'],
       ['A', 'A4']], dtype='<U69')

------------------------------
8 uniques of 'grade':


array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

------------------------------
36 uniques of 'sub_grade':


array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

In [28]:
# 1. Let's see some data related w/the '' in sub_grade

display((data_str[:,4] == '').sum())            # num of '' in sub_grade
np.argwhere(data_str[:,4] == '')                # Ixs of '' un sub_grade
display(np.argwhere(data_str[:,4] == '')[0])    # 1st appearance of ''

# display(np.unique(data_str[:,4], return_index=True, return_counts=True))
unique_arrays = np.unique(data_str[:,4], return_index=True, return_counts=True)
# display(unique_arrays)
# W/unique show only 1st el ('') of all arrays
for i in range(len(unique_arrays)):
    display(unique_arrays[i][0])
print()

# Show all rows (cols 3,4) where sub_grade (col4) = ''
print('grade', 'sub_grade')
for row in np.argwhere(data_str[:,4] == ''):
    print(data_str[row, 3:5].squeeze())

514

array([31], dtype=int64)

''

31

514


grade sub_grade
['D' '']
['B' '']
['D' '']
['B' '']
['B' '']
['D' '']
['B' '']
['F' '']
['C' '']
['D' '']
['B' '']
['G' '']
['B' '']
['' '']
['C' '']
['C' '']
['C' '']
['' '']
['D' '']
['' '']
['A' '']
['A' '']
['F' '']
['E' '']
['A' '']
['A' '']
['' '']
['D' '']
['D' '']
['C' '']
['A' '']
['A' '']
['B' '']
['A' '']
['B' '']
['D' '']
['E' '']
['A' '']
['A' '']
['B' '']
['A' '']
['B' '']
['C' '']
['C' '']
['C' '']
['C' '']
['A' '']
['C' '']
['C' '']
['B' '']
['A' '']
['B' '']
['C' '']
['C' '']
['E' '']
['' '']
['C' '']
['D' '']
['B' '']
['E' '']
['D' '']
['C' '']
['C' '']
['C' '']
['A' '']
['A' '']
['D' '']
['A' '']
['E' '']
['B' '']
['B' '']
['B' '']
['C' '']
['A' '']
['C' '']
['B' '']
['C' '']
['D' '']
['E' '']
['D' '']
['C' '']
['B' '']
['B' '']
['B' '']
['B' '']
['B' '']
['A' '']
['B' '']
['B' '']
['A' '']
['E' '']
['C' '']
['B' '']
['A' '']
['A' '']
['E' '']
['B' '']
['B' '']
['C' '']
['C' '']
['E' '']
['D' '']
['B' '']
['B' '']
['E' '']
['B' '']
['D' '']
['E' '']
['C' '']
['B' ''

In [29]:
# 2. Let's substitute the empty spaces in sub_grade with the help of grade
for gd in np.unique(data_str[:,3])[1:]:
    data_str[:,4] = np.where((data_str[:,4] == '') & (data_str[:,3] == gd),
                             gd + '5',      # risk-adverse, '5' worst case
                             data_str[:,4])

In [30]:
# 3. Let´s check w/ 1. readings.
display((data_str[:,4] == '').sum())            # num of '' in sub_grade
np.argwhere(data_str[:,4] == '')                # Ixs of '' un sub_grade
display(np.argwhere(data_str[:,4] == '')[0])    # 1st appearance of ''

# display(np.unique(data_str[:,4], return_index=True, return_counts=True))
unique_arrays = np.unique(data_str[:,4], return_index=True, return_counts=True)
# display(unique_arrays)
# W/unique show only 1st el ('') of all arrays
for i in range(len(unique_arrays)):
    display(unique_arrays[i][0])
print()

# Show all rows (cols 3,4) where sub_grade (col4) = ''
print('grade', 'sub_grade')
for row in np.argwhere(data_str[:,4] == ''):
    print(data_str[row, 3:5].squeeze())

9

array([222], dtype=int64)

''

222

9


grade sub_grade
['' '']
['' '']
['' '']
['' '']
['' '']
['' '']
['' '']
['' '']
['' '']


In [31]:
# 4. Replace the remaining '' in sub_grade w/cat H1 (worst of G5)
data_str[:,4] = np.where(data_str[:,4] == '', 'H1', data_str[:,4])
np.unique(data_str[:,4], return_index=True, return_counts=True)

(array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
        'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
        'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69'),
 array([ 102,   23,  190,   19,    1,   44,   21,   59,   33,    2,   11,    9,    0,    5,   83,
          10,   85,   30,   55,   15,   93,   76,    4,   16,   37,   34,   50,   86,    6,   87,
         208,  447, 1138, 1732,  178,  222], dtype=int64),
 array([285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267, 250,
        255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5,   9],
       dtype=int64))

In [32]:
# 5. Remove 'grade' from data and header
data_str = np.delete(data_str, 3, axis=1)
header_str = np.delete(header_str, 3)
display(data_str[:,3], header_str)

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [33]:
# 6. Replace sub_grade cats by numbers w/dic build w/zip Python function.
keys = np.unique(data_str[:,3])
# values = [i for i in range(1, np.unique(data_str[:,3]).size + 1)]
# values = list(range(1, np.unique(data_str[:,3]).size + 1))
values = np.arange(1, np.unique(data_str[:,3]).size + 1)
display (keys, values)
dic_sub_gd = dict(zip(keys, values))    # {'A1':1, 'A2':2, ...} is created
dic_sub_gd  

# for k,v in dic_sub_gd.items():
#     data_str[:,3] = np.where(data_str[:,3] == k, v, data_str[:,3])

for k in np.unique(data_str[:,3]):
    data_str[:,3] = np.where(data_str[:,3] == k,
                             dic_sub_gd[k],
                             data_str[:,3])


data_str[:,3]
np.unique(data_str[:,3])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

<a id="08"></a>
## 8. Manipulating Text Data: Verification Status & URL

- verification_status: convert to dummy 0,1 - bad, good. np.where(cond, 0,1)
- url: observe the content. Discover same url change fuinal num, strip the text, see the num = a data_num'id', arr_equal .astype int 32 True. DECIDE to ELIMINATE url (of course form data an header)
- check data_num'id' not change.

In [34]:
# 0. Let's check the headers and data
display(header_str)
display(np.where(header_str == 'verification_status'))
display(np.argwhere(header_str == 'verification_status'))

display(header_str[4], data_str[:,4])
np.unique(data_str[:,4], return_index=True, return_counts=True)

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

(array([4], dtype=int64),)

array([[4]], dtype=int64)

'verification_status'

array(['Verified', 'Source Verified', 'Verified', ..., 'Source Verified', 'Source Verified', ''],
      dtype='<U69')

(array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69'),
 array([7, 3, 1, 0], dtype=int64),
 array([ 500, 2673, 4116, 2711], dtype=int64))

In [35]:
# 1. Makes dummies: replace unique of verification_statu by 0, 1
# 1 = Good = 'Source Verified', 'Verified'; 0, Bad: '', 'Not Verified'.
data_str[:,4] = np.where((data_str[:,4] == '') |
                         (data_str[:,4] == 'Not Verified'),
                         0,
                         1)

display(header_str[4], data_str[:,4])
np.unique(data_str[:,4], return_index=True, return_counts=True)

'verification_status'

array(['1', '1', '1', ..., '1', '1', '0'], dtype='<U69')

(array(['0', '1'], dtype='<U69'),
 array([3, 0], dtype=int64),
 array([3173, 6827], dtype=int64))

In [36]:
# 2. Look url column (field)
display(np.argwhere(header_str == 'url'))
display(data_str[:,5])

array([[5]], dtype=int64)

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249'], dtype='<U69')

In [37]:
# 3. Let´s get rid of text, leave only numbers (pay attention 'loan-id=')
data_str[:,5] = np.chararray.strip(data_str[:,5],
                   'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')
display(data_str[:,5])

# loan_id idem to data_num['id]?
display(np.argwhere(header_num == 'id'))
display(data_num[:,0])

# Check comparing with the same dtype
np.array_equal(data_str[:,5].astype(np.int32), data_num[:,0].astype(np.int32))

array(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'], dtype='<U69')

array([[0]], dtype=int64)

array([48010226., 57693261., 59432726., ..., 50415990., 46154151., 66055249.])

True

In [38]:
# 4. Delete url (data and header) all data is in data_num['id']
display(header_str := np.delete(header_str, 5))
display(data_str := np.delete(data_str, 5, axis=1))

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

array([['5', '1', '36', '13', '1', 'CA'],
       ['0', '1', '36', '5', '1', 'NY'],
       ['9', '1', '36', '10', '1', 'PA'],
       ...,
       ['6', '1', '36', '5', '1', 'CA'],
       ['4', '1', '36', '17', '1', 'OH'],
       ['12', '1', '36', '4', '0', 'IL']], dtype='<U69')

In [39]:
# 5 Recheck data_num[:,0] 'id' to see the values hasn't change
data_num[:,0]

array([48010226., 57693261., 59432726., ..., 50415990., 46154151., 66055249.])

## 9. Manipulating Text Data: State Address
<a id="09"></a>

- 'addr_state' change to 'state_address: There are exactly 50 states in the USA.
- IOWA - IA is not in the data: we suspect that it was purposely left as a __*baseline benchmark*__
- (This shouldn't come as a big surprise). When doing research or analysis on a variable with many categories, it is normal to pick one as a benchmark and include dummy variables for the rest.
- With such an approach the one without a dummy variable (IA) will serve as the base case and we will either increase or decrease the rest based on their coefficients.
- We'll assume this was IA. We can also see how many times each state features in the column.
- Using argsort we sort the uniques according to the number or applicants (decreasing) from e/one.
- Most of the accounts come from highly populated and wealthy states like CA, NY, TX and FL -> The four states with more accounts than the '', => there are mor applications with missing or unreported addresses than there are for 45 of the other states.
- We have very little data for too many states to examine each one individually, then if we assign a unique val to each state, this will allow outliers to have a big influence on the coefficients to less representative states.
- The mode categories a variable has, the fewer data will be available for each one: the states with fewer applications will be more vulnerable to have their coefficients affected by outliers.
- To solve this problem, se need to group these states according to a certain common characteristics -> Geographical location: west, south, midwest, east.

In [40]:
# 0. Look the headers and content of col of interest
display(header_str)
display(np.argwhere(header_str =='addr_state'))
header_str[5] = 'state_address'
display(data_str[:,5])
# display(np.unique(data_str[:,5]))
print(f" Num of '': {data_str[data_str == ''].size}")
np.unique(data_str[:,5], return_counts=True)

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

array([[5]], dtype=int64)

array(['CA', 'NY', 'PA', ..., 'CA', 'OH', 'IL'], dtype='<U69')

 Num of '': 500


(array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
        'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
        'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
        'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69'),
 array([ 500,   26,  119,   74,  220, 1336,  201,  143,   27,   27,  690,  321,   44,  389,  152,
          84,   84,  116,  210,  222,   10,  267,  156,  160,   61,   28,  261,   16,   25,   58,
         341,   57,  130,  777,  312,   83,  108,  320,   40,  107,   24,  143,  758,   74,  242,
          17,  216,  148,   49,   27], dtype=int64))

In [41]:
# 1. Sort in crescent order or appearance
display(header_str)
st_names, st_counts = np.unique(data_str[:,5], return_counts=True)
# display(st_names, st_counts)
# display(np.sort(-st_counts))
st_counts_sort_ixs = np.argsort(-st_counts)
display(st_counts_sort_ixs)
st_names[st_counts_sort_ixs], st_counts[st_counts_sort_ixs]


array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

array([ 5, 33, 42, 10,  0, 13, 30, 11, 37, 34, 21, 26, 44, 19,  4, 46, 18,  6, 23, 22, 14, 47,  7,
       41, 32,  2, 17, 36, 39, 16, 15, 35, 43,  3, 24, 29, 31, 48, 12, 38, 25,  9,  8, 49,  1, 28,
       40, 45, 27, 20], dtype=int64)

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10], dtype=int64))

In [42]:
# 1b. JM dic method - Sort in crescent order or appearance
display(header_str)
uniq_st_add = np.unique(data_str[:,5], return_counts=True)
dic_stadd_frec = dict(zip(uniq_st_add[0], uniq_st_add[1]))
print(dic_stadd_frec)

# display(~np.sort(~uniq_st_add[1]))
display(-np.sort(-uniq_st_add[1]))
# print(sorted(uniq_st_add[1], reverse=True))
# print(sorted(dic_stadd_frec.values(), reverse=True))
# display(np.sort(dic_stadd_frec.values()))   # AxisError
# print(dic_stadd_frec.values(), type(dic_stadd_frec.values()))

dic_stadd_fcy = dict()
for count in -np.sort(-uniq_st_add[1]):
    for k,v in dic_stadd_frec.items():
        if v == count:
            dic_stadd_fcy[k] = v

print(dic_stadd_fcy)


array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

{'': 500, 'AK': 26, 'AL': 119, 'AR': 74, 'AZ': 220, 'CA': 1336, 'CO': 201, 'CT': 143, 'DC': 27, 'DE': 27, 'FL': 690, 'GA': 321, 'HI': 44, 'IL': 389, 'IN': 152, 'KS': 84, 'KY': 84, 'LA': 116, 'MA': 210, 'MD': 222, 'ME': 10, 'MI': 267, 'MN': 156, 'MO': 160, 'MS': 61, 'MT': 28, 'NC': 261, 'ND': 16, 'NE': 25, 'NH': 58, 'NJ': 341, 'NM': 57, 'NV': 130, 'NY': 777, 'OH': 312, 'OK': 83, 'OR': 108, 'PA': 320, 'RI': 40, 'SC': 107, 'SD': 24, 'TN': 143, 'TX': 758, 'UT': 74, 'VA': 242, 'VT': 17, 'WA': 216, 'WI': 148, 'WV': 49, 'WY': 27}


array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
        216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
         84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
         25,   24,   17,   16,   10], dtype=int64)

{'CA': 1336, 'NY': 777, 'TX': 758, 'FL': 690, '': 500, 'IL': 389, 'NJ': 341, 'GA': 321, 'PA': 320, 'OH': 312, 'MI': 267, 'NC': 261, 'VA': 242, 'MD': 222, 'AZ': 220, 'WA': 216, 'MA': 210, 'CO': 201, 'MO': 160, 'MN': 156, 'IN': 152, 'WI': 148, 'CT': 143, 'TN': 143, 'NV': 130, 'AL': 119, 'LA': 116, 'OR': 108, 'SC': 107, 'KS': 84, 'KY': 84, 'OK': 83, 'AR': 74, 'UT': 74, 'MS': 61, 'NH': 58, 'NM': 57, 'WV': 49, 'HI': 44, 'RI': 40, 'MT': 28, 'DC': 27, 'DE': 27, 'WY': 27, 'AK': 26, 'NE': 25, 'SD': 24, 'VT': 17, 'ND': 16, 'ME': 10}


In [43]:
# 2. Let's take care of empty vals ('') -> assign 0 to not enter any category
data_str[:,5] = np.where(data_str[:,5] == '', 0, data_str[:,5])
np.unique(data_str[:,5], return_counts=True)

(array(['0', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
        'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
        'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
        'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69'),
 array([ 500,   26,  119,   74,  220, 1336,  201,  143,   27,   27,  690,  321,   44,  389,  152,
          84,   84,  116,  210,  222,   10,  267,  156,  160,   61,   28,  261,   16,   25,   58,
         341,   57,  130,  777,  312,   83,  108,  320,   40,  107,   24,  143,  758,   74,  242,
          17,  216,  148,   49,   27], dtype=int64))

In [44]:
# 3. Group the states according geographical location.
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

cnt = 1
for region in (states_west, states_south, states_midwest, states_east):
    data_str[:,5] = np.where(np.isin(data_str[:,5], region), cnt, data_str[:,5])
    cnt += 1

np.unique(data_str[:,5], return_counts=True)

(array(['0', '1', '2', '3', '4'], dtype='<U69'),
 array([ 500, 2467, 3384, 1733, 1916], dtype=int64))

## 10. Manipulating Text Data: Converting Strings and Creating a Checkpoint
<a id="10"></a>

- We have all data_str vals in numeric fashion but still are str, then que need to convert

In [45]:
# 0. Let's look the data
data_str

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

In [46]:
# 1. Cast to numeric type (int)
data_str = data_str.astype(int)
display(data_str)
show_attr('data_str')

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

' data_str: | shape: (10000, 6) | ndim: 2 | size: 60000 | dtype: int32 '

In [47]:
# 2. Create the checkpoint and check it
chkpt_str = chkpt('chkpt-str', header_str, data_str)
display(chkpt_str['header'], chkpt_str['data'])
np.array_equal(chkpt_str['data'], data_str)

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

True

## 11. Manipulating Numeric Data: Substitute Filler Values
<a id="11"></a>

- Here we have to recall (remember) the temp_fill and orig_stats we calc early
- It suppose 'id' won't have missings (temp_fill) <- check>
#### Rest of cols_num will be filled w/min and max depending:
- 'loan_amnt': 1,000 - 35,000. Worst -> Max. Can be of any size according to the Bank's current abilities.
- Loan Amount: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value (El importe del préstamo solicitado por el prestatario. Si en algún momento el departamento de crédito reduce el importe del préstamo, esto se reflejará en este valor.)

- 'int_rate': 6 - 28.99. Worst -> Max. Varies according to the loan amount and credit risk status of the individual or business taking the loan.
- Interest Rate: Interest Rate on the loan (Tasa de interés del préstamo)

- 'installment': 31.42 - 1,372.97. Worst -> Max. The number and size of the installments will depend on the contract sign.
- Installment (Cuota): The monthly payment owed by the borrower if the loan originates (El pago mensual adeudado por el prestatario si se origina el préstamo.)

- 'total_pymnt': 0 - 41,913.62. Will obviously incorporate all parameters mentioned above and will certainly be the largest positive value of all in nominal terms.
- Total payment: Payments received to date for total amount funded (Pagos recibidos hasta la fecha por el monto total financiado)

- 'funded_amnt': 1,000 - 35,000. Worst -> Min. Is the only col for which we want to set the filler values equal to the minimum.
- Funded amount (Monto financiado): The total amount committed to that loan at that point in time. (El monto total comprometido para ese préstamo en ese momento.)

because they are assuming the worst case scenario

for funded amount = the amount of money that the person provided to pay his loan, so if you only paid minimum amount, that means you still have  a lot more to pay and so it's considered bad scenario to have a minimum value.

the others use maximum cause it again represents  the worst scenario, loan amount for example, for the bank,it's better to have someone who only took 10k insted of having someone who took 200k. the 200k person is more likely to default than the other one.

In [48]:
# 0. Let's look numeric header and data and old calcs.
display(header_num, data_num)
display(temp_fill, orig_stats)
print('temp_fill:', show_thseps(temp_fill))
display('Only Numeric orig_stats:', orig_stats[:,cols_num])

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

68616520.0

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

temp_fill: 68,616,520.0


'Only Numeric orig_stats:'

array([[  373332.  ,     1000.  ,     1000.  ,        6.  ,       31.42,        0.  ],
       [54015809.19,    15273.46,    15311.04,       16.62,      440.92,     3143.85],
       [68616519.  ,    35000.  ,    35000.  ,       28.99,     1372.97,    41913.62]])

In [49]:
# 1. Check that 'id' won't have missings (temp_fill)
display(np.argwhere(header_num == 'id'))
display(np.where(data_num[:,0] == temp_fill))
# display(data_num[:,0] == temp_fill)
# display((data_num[:,0] == temp_fill).sum())
# display(np.sum(data_num[:,0] == temp_fill))
display(np.isin(data_num[:,0], temp_fill))
# display(np.isin(temp_fill, data_num[:,0]))
display(np.isin(data_num[:,0], temp_fill).sum())

array([[0]], dtype=int64)

(array([], dtype=int64),)

array([False, False, False, ..., False, False, False])

0

In [50]:
# 2. Replace temp_fill for min in 'funded_amnt'
display(header_num)
display(np.argwhere(header_num == 'funded_amnt'))
display(np.argwhere(header_full == 'funded_amnt'))
display(cols_num)
display(orig_stats[:,cols_num])
display(orig_stats[0, cols_num[2]])
display(np.isin(data_num[:,2], temp_fill).sum())

# np.min(np.where(data_num[:,2] == temp_fill,
#                 orig_stats[0, cols_num[2]],
#                 data_num[:,2]))

data_num[:,2] = np.where(data_num[:,2] == temp_fill,
                         orig_stats[0, cols_num[2]],
                         data_num[:,2])

display(np.min(data_num[:,2]))
display(np.isin(data_num[:,2], temp_fill).sum())

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

array([[2]], dtype=int64)

array([[4]], dtype=int64)

array([ 0,  2,  4,  6,  7, 13], dtype=int64)

array([[  373332.  ,     1000.  ,     1000.  ,        6.  ,       31.42,        0.  ],
       [54015809.19,    15273.46,    15311.04,       16.62,      440.92,     3143.85],
       [68616519.  ,    35000.  ,    35000.  ,       28.99,     1372.97,    41913.62]])

1000.0

500

1000.0

0

In [51]:
# 3. Replace temp_fill for max in others num columns
display(header_num)
display(cols_num)

# ix[1]='loan_amnt'; ix[3]='int_rate'; ix[4]=...
for i in (1,3,4,5):
    data_num[:,i] = np.where(data_num[:,i] == temp_fill,
                             orig_stats[2, cols_num[i]],
                             data_num[:,i])
    
display(data_num)
display(np.max(data_num), temp_fill)
print(show_thseps(np.max(data_num)), show_thseps(temp_fill))
print('temp_fill - np.max(data_num):', temp_fill - np.max(data_num))


array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

array([ 0,  2,  4,  6,  7, 13], dtype=int64)

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  ,       28.99,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,       28.99,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,       28.99,     1372.97,     2185.64],
       [46154151.  ,    35000.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,       28.99,      309.97,      301.9 ]])

68616519.0

68616520.0

68,616,519.0 68,616,520.0
temp_fill - np.max(data_num): 1.0


## 12. Manipulating Numeric Data: Currency Change - The Exchange Rate

- European office and USA BANK: we must keep all records in EUR as well as USD 
- To convert USD to EU we need the exchange rate between the two currencies at the time of each loan application.
- issue_date were just months rather than specific dates. 
- EUR-USD.csv: contains the avg monthly exchange rate for 2015
- Open, High, Low, Close, Volume columns. Close for us -> Adjusted closing prices.
- In analysis, we often prefer to use the adjusted closing price.
- We're no using a constant (unique) exchange rate (actually wue have one for each month), so we'll create a new col to store the exchange rates for each account (link with issue_date)

In [52]:
# 0. Let's see the content of EUR-USD.csv
eur_usd_full = np.genfromtxt('9_EUR-USD.csv',
                             delimiter=',',
                             dtype=str,
                             autostrip=True)

# eur_usd_full <- Only care 'Close column'


In [53]:
# 1. Import the data we really care of EUR-USD.csv (Close, col[3])
eur_usd = np.genfromtxt('9_EUR-USD.csv',
                        delimiter=',',
                        autostrip=True,
                        usecols=3,
                        skip_header=1)
display(eur_usd)
show_attr('eur_usd')    # 12 vals, one for e/month (1-12) Ixs 0-11

array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

' eur_usd: | shape: (12,) | ndim: 1 | size: 12 | dtype: float64 '

In [54]:
# 2. Create exchange rate col (1st doing equal to issue_date)
display(header_str)
display(np.argwhere(header_str == 'issue_date'))
display(exch_rate := data_str[:,0])

# For e/account set the exch_rate depending on issue_date
for i in range(1,13):     # for e/month 1-12
    exch_rate = np.where(exch_rate == i, eur_usd[i - 1], exch_rate)

# For cases val 0, missing vals in issue_date use annual mean
exch_rate = np.where(exch_rate == 0, np.mean(eur_usd), exch_rate)

display(exch_rate, show_attr('exch_rate'))

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

array([[0]], dtype=int64)

array([ 5,  0,  9, ...,  6,  4, 12])

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

' exch_rate: | shape: (10000,) | ndim: 1 | size: 10000 | dtype: float64 '

In [55]:
# 3. Add this col to the dataset (hstack)
display(data_num, show_attr('data_num'))

exch_rate = exch_rate.reshape(10000,1)
display(exch_rate, show_attr('exch_rate'))

data_num = np.hstack((data_num, exch_rate))
display(data_num, show_attr('data_num'))


array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  ,       28.99,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,       28.99,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,       28.99,     1372.97,     2185.64],
       [46154151.  ,    35000.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,       28.99,      309.97,      301.9 ]])

' data_num: | shape: (10000, 6) | ndim: 2 | size: 60000 | dtype: float64 '

array([[1.1 ],
       [1.11],
       [1.13],
       ...,
       [1.12],
       [1.11],
       [1.09]])

' exch_rate: | shape: (10000, 1) | ndim: 2 | size: 10000 | dtype: float64 '

array([[48010226.  ,    35000.  ,    35000.  , ...,     1184.86,     9452.96,        1.1 ],
       [57693261.  ,    30000.  ,    30000.  , ...,      938.57,     4679.7 ,        1.11],
       [59432726.  ,    15000.  ,    15000.  , ...,      494.86,     1969.83,        1.13],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     1372.97,     2185.64,        1.12],
       [46154151.  ,    35000.  ,    10000.  , ...,      354.3 ,     3199.4 ,        1.11],
       [66055249.  ,    10000.  ,    10000.  , ...,      309.97,      301.9 ,        1.09]])

' data_num: | shape: (10000, 7) | ndim: 2 | size: 70000 | dtype: float64 '

In [56]:
# 4. Add a value to header_num (Since we add a col to data_num) - concatenate()
display(header_num, show_attr('header_num'))
header_num = np.concatenate((header_num, np.array(['exchange_rate'])))
display(header_num, show_attr('header_num'))

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

' header_num: | shape: (6,) | ndim: 1 | size: 6 | dtype: <U19 '

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

' header_num: | shape: (7,) | ndim: 1 | size: 7 | dtype: <U19 '

## 13. Manipulating Numeric Data: Currency Change - From USD to EUR

- We'll create 3 new cols with the values in EUR
- Also we'll accommodate thats new cols aside the corresponding ones in USD.
- We'll rename de cols with _EUR the new ones and _USD the olds

#### Converson from USD to EUR
- We have the exchange EUR-USD rate, thats means:
- EUR / USD = rate in the file => __USD = EUR / rate__ (USD expressed in EUR)

In [57]:
# 0. Let's see which cols in headers (num because they have amnt in usd)
display(header_num, show_attr('header_num'))

# USD cols: loan_amnt, founded_amnt, installment, total_pymnt -> cols_usd
display(np.unique(header_num, return_index=True))
cols_usd = np.array([1,2,4,5])            # usd-cols Ixs
display(header_num[cols_usd])

# Data cols we are interested:
display(data_num[:,cols_usd], show_attr('data_num[:,cols_usd]'))

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

' header_num: | shape: (7,) | ndim: 1 | size: 7 | dtype: <U19 '

(array(['exchange_rate', 'funded_amnt', 'id', 'installment', 'int_rate', 'loan_amnt', 'total_pymnt'],
       dtype='<U19'),
 array([6, 2, 0, 4, 3, 1, 5], dtype=int64))

array(['loan_amnt', 'funded_amnt', 'installment', 'total_pymnt'], dtype='<U19')

array([[35000.  , 35000.  ,  1184.86,  9452.96],
       [30000.  , 30000.  ,   938.57,  4679.7 ],
       [15000.  , 15000.  ,   494.86,  1969.83],
       ...,
       [10000.  , 10000.  ,  1372.97,  2185.64],
       [35000.  , 10000.  ,   354.3 ,  3199.4 ],
       [10000.  , 10000.  ,   309.97,   301.9 ]])

' data_num[:,cols_usd]: | shape: (10000, 4) | ndim: 2 | size: 40000 | dtype: float64 '

In [58]:
# 1. Create the new data_num cols that will have the values in EUR
display(data_num[:,6], show_attr('data_num[:,6]'))    # exchange rates col

for ix in cols_usd:     # for e/usd original col 
    data_num = np.hstack((data_num,
                          np.reshape(data_num[:,ix] / data_num[:,6], (10000,1))))
    
display(data_num, show_attr('data_num'))


array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

' data_num[:,6]: | shape: (10000,) | ndim: 1 | size: 10000 | dtype: float64 '

array([[48010226.  ,    35000.  ,    35000.  , ...,    31933.3 ,     1081.04,     8624.69],
       [57693261.  ,    30000.  ,    30000.  , ...,    27132.46,      848.86,     4232.39],
       [59432726.  ,    15000.  ,    15000.  , ...,    13326.3 ,      439.64,     1750.04],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     8910.3 ,     1223.36,     1947.47],
       [46154151.  ,    35000.  ,    10000.  , ...,     8997.4 ,      318.78,     2878.63],
       [66055249.  ,    10000.  ,    10000.  , ...,     9145.8 ,      283.49,      276.11]])

' data_num: | shape: (10000, 11) | ndim: 2 | size: 110000 | dtype: float64 '

In [59]:
# 2. Modify the header to contain the new columns:
# same name as originals + _EUR, ex. header_col[8] = header_col[1] + _EUR
display(header_num, show_attr('header_num'))
header_eur = np.array([col_nm + '_EUR' for col_nm in header_num[cols_usd]])
display(header_eur, show_attr('header_eur'))
# header_num = np.hstack((header_num, header_eur))      # JM Same result
header_num = np.concatenate((header_num, header_eur))
display(header_num, show_attr('header_num'))

# for ix in cols_usd:     # JM way 1
#     header_num = np.hstack((header_num,
#                             np.array(header_num[ix] + '_EUR')))

# for ix in cols_usd:     # JM way 2
#     header_num = np.concatenate((header_num,
#                                  np.array([header_num[ix] + '_EUR'])))

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

' header_num: | shape: (7,) | ndim: 1 | size: 7 | dtype: <U19 '

array(['loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U15')

' header_eur: | shape: (4,) | ndim: 1 | size: 4 | dtype: <U15 '

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate',
       'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U19')

' header_num: | shape: (11,) | ndim: 1 | size: 11 | dtype: <U19 '

In [60]:
# 3. Rename the cols names that te amnt in usd sey _USD
display(header_num[cols_usd])
header_num[cols_usd] = np.array(
    [usd_nm + '_USD' for usd_nm in header_num[cols_usd]])
display(header_num)

array(['loan_amnt', 'funded_amnt', 'installment', 'total_pymnt'], dtype='<U19')

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rate', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

In [61]:
# 4. Rearrange the cols position to have side by side USD follow by EUR (header & data)
display(header_num)
display(np.argwhere(header_num).squeeze())
cols_ixs_order = [0, 1, 7, 2, 8, 3, 4, 9, 5, 10, 6]
header_num = header_num[cols_ixs_order]
display(header_num, show_attr('header_num'))

data_num = data_num[:,cols_ixs_order]
display(data_num, show_attr('data_num'))

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rate', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

' header_num: | shape: (11,) | ndim: 1 | size: 11 | dtype: <U19 '

array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
       [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
       [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
       [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
       [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]])

' data_num: | shape: (10000, 11) | ndim: 2 | size: 110000 | dtype: float64 '

## Numeric data preprocess summary:
10. Appropriately filled out any missing values
11. Added exchange rates for e/applicant (account)
12. Created EUR version of the 4 monetary values

In [62]:
# 5. 'int_rate' col: better to have values between 0 and 1
display(header_num)
display(np.argwhere(header_num == 'int_rate'))
display(data_num[:,5], show_attr('data_num[:,5]'))
data_num[:,5] = data_num[:,5] / 100
display(data_num[:,5], show_attr('data_num[:,5]'))
# np.unique(data_num[:,5])
print(f'Min: {np.min(data_num[:,5])} - Max: {np.max(data_num[:,5])}')

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

array([[5]], dtype=int64)

array([13.33, 28.99, 28.99, ..., 28.99, 16.55, 28.99])

' data_num[:,5]: | shape: (10000,) | ndim: 1 | size: 10000 | dtype: float64 '

array([0.13, 0.29, 0.29, ..., 0.29, 0.17, 0.29])

' data_num[:,5]: | shape: (10000,) | ndim: 1 | size: 10000 | dtype: float64 '

Min: 0.06 - Max: 0.2899


In [63]:
# 6 Create checkpoing of numeric data preprocess
chkpt_num = chkpt('chkpt-num', header_num, data_num)
display(chkpt_num.files)

display(chkpt_num['header'], show_attr("chkpt_num['header']"))
display(chkpt_num['data'], show_attr("chkpt_num['data']"))
display(np.array_equal(chkpt_num['header'], header_num))
display(np.array_equal(chkpt_num['data'], data_num))

['header', 'data']

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

" chkpt_num['header']: | shape: (11,) | ndim: 1 | size: 11 | dtype: <U19 "

array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
       [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
       [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
       [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
       [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]])

" chkpt_num['data']: | shape: (10000, 11) | ndim: 2 | size: 110000 | dtype: float64 "

True

True

## 14. Completing the Dataset:

- Finalize de dataset. Put toeghether all the parts and save te preprocessed data in a .csv file
- At the begining we split the orig dtset in two num and str (also header and data of both).
- Now we have these parts: header_str, data_str, header_num, data_num
- We construct one of all and use the checkpoints files cause they are safer if we modify de arrays latter to save the checkpoints

In [64]:
# 0. let´s take a look

for part in ('header', 'data'):
    display(chkpt_str[part], show_attr(f"chkpt_str['{part}']"))
for part in ('header', 'data'):   
    display(chkpt_num[part], show_attr(f"chkpt_num['{part}']"))


array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

" chkpt_str['header']: | shape: (6,) | ndim: 1 | size: 6 | dtype: <U19 "

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

" chkpt_str['data']: | shape: (10000, 6) | ndim: 2 | size: 60000 | dtype: int32 "

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

" chkpt_num['header']: | shape: (11,) | ndim: 1 | size: 11 | dtype: <U19 "

array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
       [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
       [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
       [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
       [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]])

" chkpt_num['data']: | shape: (10000, 11) | ndim: 2 | size: 110000 | dtype: float64 "

In [65]:
# 1. Let's hstack datas (same dim and same num of rows) num + str
data = np.hstack((chkpt_num['data'], chkpt_str['data']))
display(data, show_attr('data'))

array([[48010226.  ,    35000.  ,    31933.3 , ...,       13.  ,        1.  ,        1.  ],
       [57693261.  ,    30000.  ,    27132.46, ...,        5.  ,        1.  ,        4.  ],
       [59432726.  ,    15000.  ,    13326.3 , ...,       10.  ,        1.  ,        4.  ],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,        5.  ,        1.  ,        1.  ],
       [46154151.  ,    35000.  ,    31490.9 , ...,       17.  ,        1.  ,        3.  ],
       [66055249.  ,    10000.  ,     9145.8 , ...,        4.  ,        0.  ,        3.  ]])

' data: | shape: (10000, 17) | ndim: 2 | size: 170000 | dtype: float64 '

In [66]:
# 2. Let's concatenate headers (jm-hstack also) num + str
header_jm = np.hstack((chkpt_num['header'], chkpt_str['header']))
display(header_jm, show_attr('header_jm'))

header = np.concatenate((chkpt_num['header'], chkpt_str['header']))
display(header, show_attr('header'))

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate',
       'issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

' header_jm: | shape: (17,) | ndim: 1 | size: 17 | dtype: <U19 '

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate',
       'issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

' header: | shape: (17,) | ndim: 1 | size: 17 | dtype: <U19 '

In [67]:
# 3. Sort the num data by id column
display(np.argwhere(header == 'id'))
display(data[:,0], np.sort(data[:,0]))
display(col0_ixs := np.argsort(data[:,0]))

# Rearrange data sorted by id (col[0])
data = data[np.argsort(data[:,0]), :]   # =data[np.argsort(data[:,0])]
display(data[:,0], np.argsort(data[:,0]))
display(data, show_attr('data'))

array([[0]], dtype=int64)

array([48010226., 57693261., 59432726., ..., 50415990., 46154151., 66055249.])

array([  373332.,   575239.,   707689., ..., 68614880., 68615915., 68616519.])

array([2086, 4812, 2353, ..., 4935, 9388, 8415], dtype=int64)

array([  373332.,   575239.,   707689., ..., 68614880., 68615915., 68616519.])

array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=int64)

array([[  373332.  ,     9950.  ,     9038.08, ...,       21.  ,        0.  ,        1.  ],
       [  575239.  ,    12000.  ,    10900.2 , ...,       25.  ,        1.  ,        2.  ],
       [  707689.  ,    10000.  ,     8924.3 , ...,       13.  ,        1.  ,        0.  ],
       ...,
       [68614880.  ,     5600.  ,     5121.65, ...,        8.  ,        1.  ,        1.  ],
       [68615915.  ,     4000.  ,     3658.32, ...,       10.  ,        1.  ,        2.  ],
       [68616519.  ,    21600.  ,    19754.93, ...,        3.  ,        0.  ,        2.  ]])

' data: | shape: (10000, 17) | ndim: 2 | size: 170000 | dtype: float64 '

In [68]:
# 4. Let's vstack headers and data
data_prepro= np.vstack((header, data))
display(data_prepro, show_attr('data_prepro'))


array([['id', 'loan_amnt_USD', 'loan_amnt_EUR', ..., 'sub_grade', 'verification_status',
        'state_address'],
       ['373332.0', '9950.0', '9038.082814338286', ..., '21.0', '0.0', '1.0'],
       ['575239.0', '12000.0', '10900.20037910145', ..., '25.0', '1.0', '2.0'],
       ...,
       ['68614880.0', '5600.0', '5121.647851612413', ..., '8.0', '1.0', '1.0'],
       ['68615915.0', '4000.0', '3658.319894008867', ..., '10.0', '1.0', '2.0'],
       ['68616519.0', '21600.0', '19754.927427647883', ..., '3.0', '0.0', '2.0']], dtype='<U32')

' data_prepro: | shape: (10001, 17) | ndim: 2 | size: 170017 | dtype: <U32 '

In [69]:
# 4. Save the .csv file
np.savetxt('loan-data-preprocessed.csv',
           data_prepro,
           delimiter=',',
           fmt='%s')

- Setting Up: Introduction to the Practical Example
- Setting Up: Importing the Data Set
- Setting Up: Checking for Incomplete Data
- Setting Up: Splitting the Dataset
- Setting Up: Creating Checkpoints
- Manipulating Text Data: Issue Date
- Manipulating Text Data: Loan Status and Term
- Manipulating Text Data: Grade and Sub Grade
- Manipulating Text Data: Verification Status & URL
- Manipulating Text Data: State Address
- Manipulating Text Data: Converting Strings and Creating a Checkpoint
- Manipulating Numeric Data: Substitute Filler Values
- Manipulating Numeric Data: Currency Change - The Exchange Rate
- Manipulating Numeric Data: Currency Change - From USD to EUR

# Introduction

Welcome to the documentation.

## Installation

To install the package, follow these steps:

1. Download the package.
2. Run the installer.

Go to Installation



## JM Walrus Operator Discover

In [70]:
## Walrus Operator NOT working in all cases - problem with Indexing
display(c := 5 + 4)
# display(data_str[:,2] := np.chararray.strip(data_str[:,2], ' months')) # SyntaxError
display(header_full_jm)
# display(header_full_jm[2] := 'term_months')
display(a_variable := 'term_months')
display(lst := [i for i in range(9) if i % 2 != 0])
# display(lst[1] := 0)    # SyntaxError

9

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U69')

'term_months'

[1, 3, 5, 7]

In [71]:
# Internal links

# For internal (within the same notebook) links use this code:
# [section title](#section-title)
# For the text in the parentheses, replace spaces and special characters with a hyphen.

# Alternatively, you can add an ID for a section right above the section title.
# Use this code: <a id="section_ID"></a>

# Make sure that the section_ID is unique within the notebook.
# Use this code: [section title](#section_ID)